In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [20]:
#board mappings
pieces = {
    '0': {
        'name': 'Space'
    },
    '1': {
        'name': 'Rook',
        'base': -1,
        'value': 5
    },
    '2': {
        'name': 'Knight',
        'base': -1,
        'value': 3
    },
    '3': {
        'name': 'Bishop',
        'base': -1,
        'value': 3
    },
    '4': {
        'name': 'Queen',
        'base': -1,
        'value': 9
    },
    '5': {
        'name': 'King',
        'base': -1,
        'value': 10
    },
    '6': {
        'name': 'Bishop',
        'base': -1,
        'value': 3
    },
    '9': {
        'name': 'BPawn',
        'base': -1,
        'value': 1
    },
    '11': {
        'name': 'Rook',
        'base': 1,
        'value': 5
    },
    '12': {
        'name': 'Knight',
        'base': 1,
        'value': 3
    },
    '13': {
        'name': 'Bishop',
        'base': 1,
        'value': 3
    },
    '14': {
        'name': 'Queen',
        'base': 1,
        'value': 9
    },
    '15': {
        'name': 'King',
        'base': 1,
        'value': 10
    },
    '16': {
        'name': 'Bishop',
        'base': 1,
        'value': 3
    },
    '10': {
        'name': 'WPawn',
        'base': 1,
        'value': 1
    },
}


def rook(index, weight=1.0, current_board=np.zeros((8, 8), dtype=np.float)):
    """
    Calculates the area of effect and weight of effect for a Rook at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :param current_board: The current chess board as a 8x8 matrix. This is used to calculate piece limits
    :return: 8x8 Matrix with piece weights.
    """
    baseline = np.zeros((8, 8), dtype=np.float)
    y = index[0]
    x = index[1]
    # left row
    for i in range(x - 1, -1, -1):
        baseline[y, i] = weight
        if current_board[y, i] != 0:
            break
    # right row
    for i in range(x + 1, 8):
        baseline[y, i] = weight
        if current_board[y, i] != 0:
            break
    # top column
    for i in range(y - 1, -1, -1):
        baseline[i, x] = weight
        if current_board[i, x] != 0:
            break
    # bottom column
    for i in range(y + 1, 8):
        baseline[i, x] = weight
        if current_board[i, x] != 0:
            break
    baseline[y, x] = 0
    return baseline


def bishop(index, weight=1.0, current_board=np.zeros((8, 8), dtype=np.float)):
    """
    Calculates the area of effect and weight of effect for a Bishop at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :param current_board: The current chess board as a 8x8 matrix. This is used to calculate piece limits
    :return: 8x8 Matrix with piece weights.
    """
    baseline = np.zeros((8, 8), dtype=np.float)
    y = index[0]
    x = index[1]
    # upper left
    for i in range(0, 8):
        try:
            x1 = x - i
            y1 = y - i
            if (x1 == x) and (y1 == y):
                continue
            if (x1 >= 0) and (y1 >= 0):
                baseline[y1, x1] = weight
                # print(baseline)
                if current_board[y1, x1] != 0:
                    break
        except IndexError:
            pass
    # lower left
    for i in range(0, 8):
        try:
            x1 = x - i
            y1 = y + i
            if (x1 == x) and (y1 == y):
                continue
            if (x1 >= 0) and (y1 >= 0):
                baseline[y1, x1] = weight
                # print(baseline)
                if current_board[y1, x1] != 0:
                    break
        except IndexError:
            pass
    # lower right
    for i in range(0, 8):
        try:
            x1 = x + i
            y1 = y + i
            if (x1 == x) and (y1 == y):
                continue
            if (x1 >= 0) and (y1 >= 0):
                baseline[y1, x1] = weight
                # print(baseline)
                if current_board[y1, x1] != 0:
                    break
        except IndexError:
            pass
    # upper right
    for i in range(0, 8):
        try:
            x1 = x + i
            y1 = y - i
            if (x1 == x) and (y1 == y):
                continue
            if (x1 >= 0) and (y1 >= 0):
                baseline[y1, x1] = weight
                # print(baseline)
                if current_board[y1, x1] != 0:
                    break
        except IndexError:
            pass
    baseline[y, x] = 0
    return baseline


def queen(index, weight=1.0, current_board=np.zeros((8, 8), dtype=np.float)):
    """
    Calculates the area of effect and weight of effect for a Queen at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :param current_board: The current chess board as a 8x8 matrix. This is used to calculate piece limits
    :return: 8x8 Matrix with piece weights.
    """
    baseline = bishop(index, weight, current_board)
    baseline += rook(index, weight, current_board)
    return baseline


def king(index, weight=1.0):
    """
    Calculates the area of effect and weight of effect for a King at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :return: 8x8 Matrix with piece weights.
    """
    baseline = np.zeros((8, 8), dtype=np.float)
    y = index[0]
    x = index[1]
    try:
        x1 = x + 1
        y1 = y
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x + 1
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x + 1
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x - 1
        y1 = y - 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x
        y1 = y - 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x - 1
        y1 = y
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x - 1
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        x1 = x + 1
        y1 = y - 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    baseline[y, x] = 0
    return baseline


def pawn(index, weight=1.0, color='WPawn'):
    """
    Calculates the area of effect and weight of effect for a King at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :param color: Expected to be WPawn or BPawn. Used to determine the orientation of the pawn
    :return: 8x8 Matrix with piece weights.
    """
    baseline = np.zeros((8, 8), dtype=np.float)
    x = index[0]
    y = index[1]
    if color == "WPawn":
        try:
            x1 = x - 1
            y1 = y + 1
            if (x1 >= 0) and (y1 >= 0):
                baseline[x1, y1] = weight
        except IndexError:
            pass
        try:
            x1 = x - 1
            y1 = y - 1
            if (x1 >= 0) and (y1 >= 0):
                baseline[x1, y1] = weight
        except IndexError:
            pass
    else:
        try:
            x1 = x + 1
            y1 = y - 1
            if (x1 >= 0) and (y1 >= 0):
                baseline[x1, y1] = weight
        except IndexError:
            pass
        try:
            x1 = x + 1
            y1 = y + 1
            if (x1 >= 0) and (y1 >= 0):
                baseline[x1, y1] = weight
        except IndexError:
            pass
    baseline[y, x] = 0
    return baseline


def knight(index, weight=1.0):
    """
    Calculates the area of effect and weight of effect for a King at a particular
    index on the board.
    :param index: The position of the piece on the board
    :param weight: The weight the piece can put on the board
    :return: 8x8 Matrix with piece weights.
    """
    baseline = np.zeros((8, 8), dtype=np.float)
    y = index[0]
    x = index[1]
    try:
        # bottom right
        x1 = x + 1
        y1 = y + 2
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        # bottom left
        x1 = x - 1
        y1 = y + 2
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass

    try:
        # upper right
        x1 = x + 1
        y1 = y - 2
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    try:
        # upper left
        x1 = x - 1
        y1 = y - 2
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    # right top
    try:
        x1 = x + 2
        y1 = y - 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    # right bottom
    try:
        x1 = x + 2
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass

    # left top
    try:
        x1 = x - 2
        y1 = y - 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    # left bottom
    try:
        x1 = x - 2
        y1 = y + 1
        if (x1 >= 0) and (y1 >= 0):
            baseline[y1, x1] = weight
    except IndexError:
        pass
    baseline[y, x] = 0
    return baseline

In [21]:
queen((4,4))

array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1.]])